<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Deep-Learning-Model-Visualization/blob/main/Chapter03/DLMV_Chapter03_06_Altair_ActivationFunctions.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Chapter 03 - 06 - Exploring activation functions using Altair

## Installation

In [1]:
# install or upgrade
!pip install --no-cache-dir --upgrade altair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2


## Imports

In [2]:
import altair as alt
import pandas as pd
import ipywidgets as widgets
import numpy as np

In [3]:
print(alt.__version__)
print(widgets.__version__)

5.3.0
7.7.1


## Maths for common activation functions

1. Sigmoid
$$
\sigma(x) = \frac{1}{(1+e^{-x})} \\
\sigma'(x) = \sigma(x)\times(1-\sigma(x))
$$

2. Tanh
$$
\tanh(x) = \frac{1}{(1+e^{-2x})} - 1 \\
\tanh'(x) = 1 - tanh^2(x)
$$

3. ReLU
$$
ReLU(x) =   \begin{cases}
              x & \text{ if } x\ge 0 \\
              0 & \text{ if } x\lt 0
            \end{cases} \\
            ReLU'(x) =   \begin{cases}
              1 & \text{ if } x\ge 0 \\
              0 & \text{ if } x\lt 0
            \end{cases}
$$

## Python implementations for common activation functions

In [4]:
def Sigmoid(x):
  return 1/(1+np.exp(-x))

def dSigmoid(x):
  return Sigmoid(x)*(1-Sigmoid(x))

def Tanh(x):
  return 2/(1+np.exp(-2*x)) - 1

def dTanh(x):
  return 1 - Tanh(x)**2

def ReLU(x):
  return np.where(x>=0, x, 0)

def dReLU(x):
  return np.where(x>=0, 1, 0)

## Helper to generate data

In [5]:
def gen_data(x, fx, dfx):
  df = pd.DataFrame({"x": x, "fx": fx(x), "dfx": dfx(x)})
  df = df.melt(id_vars    = ["x"],
               var_name   = "type",
               value_name = "y")
  return df

## Visualization

In [6]:
def plot(source, title):
  # Create a base chart and set the data source
  base = alt.Chart(source)
  # Layer 1: The lines
  lines = base.mark_line().encode(
      x='x:Q', y='y:Q', color='type:N'
  )
  # Layer 2: Transparent selectors across the chart
  nearest = alt.selection_point(nearest = True,
                                on      = 'mouseover',
                                fields  = ['x'],
                                empty   = False)
  selectors = base.mark_point().encode(
      x = 'x:Q', opacity = alt.value(0),
  ).add_params(nearest)
  # Layer 3: Points on the lines for selcted x
  points = lines.mark_point().encode(
      opacity=alt.condition(
          nearest,
          alt.value(1),
          alt.value(0))
  )
  # Layer 4: Text labels for the points
  text = lines.mark_text(align='left', dx=5, dy=-5).encode(
    text = alt.condition(
        nearest,
        'y:Q', alt.value(' '), format='.3f'))
  # Layer 5: A rule at the location of the selection
  rules = base.mark_rule(color='gray').encode(
      x='x:Q',
  ).transform_filter(nearest)
  # Put the five layers into a chart and set properties
  fig = alt.layer(
      lines, selectors, points, rules, text
  ).properties(
      width=200, height=200, title=title
  )
  return fig

In [7]:
x = np.linspace(-6, 6, 100)
sigmoid = plot(gen_data(x, Sigmoid, dSigmoid), "Sigmoid")
tanh = plot(gen_data(x, Tanh, dTanh), "Tanh")
relu = plot(gen_data(x, ReLU, dReLU), "ReLU")
fig = alt.hconcat(sigmoid, tanh, relu)
fig

alt.HConcatChart(...)

## Save and donwload saved file

In [8]:
filename = "activations.html"
fig.save(filename)

In [9]:
from google.colab import files
files.download("activations.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>